In [107]:
loss,accuracy=full_model.evaluate(test)

In [108]:
print("Accuracy",accuracy)

Accuracy 0.9564032554626465


In [43]:
loss,accuracy=model1.evaluate(test)



37/37 [==============================] - 28s 605ms/step - loss: 0.3049 - sparse_categorical_accuracy: 0.9264


In [44]:

print("Accuracy",accuracy)



Accuracy 0.9264305233955383


---
## model1 summary, flops

In [62]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten

In [63]:
def get_flops(model, batch_size=None):
    if batch_size is None:
        batch_size = 1

    real_model = tf.function(model).get_concrete_function(tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype))
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops

In [66]:
def main(batch_size):
    model1.summary()

    flops = get_flops(model1, batch_size)
    print(f"FLOPS: {flops}")
    return model1

In [67]:
m = main(16)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 moat (MOAT)                 {'stage1': (None, 128, 1  4809320   
                             28, 40),                            
                              'res1': (None, 128, 128            
                             , 40),                              
                              'stage2': (None, 64, 64            
                             , 40),                              
                              'res2': (None, 64, 64,             
                             40),                                
                              'stage3': (None, 32, 32            
                             , 80),                              
                              'res3': (None, 32, 32,       

---
### full_model summary, flops

In [87]:
def main(batch_size):
    full_model.summary()

    flops = get_flops(full_model, batch_size)
    print(f"FLOPS: {flops}")
    return full_model

In [88]:
m = main(16)
# FLOPS = 32 * 32 * 3 (= 3072) * 10 * 2 + 10 = 61450

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 moat3 (MOAT)                   {'stage1': (None, 1  4809320     ['input_16[0][0]']               
                                28, 128, 40),                                                     
                                 'res1': (None, 128                                               
                                , 128, 40),                                                       
                                 'stage2': (None, 6                                         